In [66]:
from pulp import LpProblem, LpVariable, LpBinary, lpSum, LpMaximize, LpStatus, PULP_CBC_CMD
from datetime import datetime, timedelta
import pandas as pd

In [67]:
# extract data
courses = pd.read_csv('courses.csv')
venues = pd.read_csv('venues.csv')

# drop unavailable venues
available_venues = venues[venues['isAvailable'] == 'yes']

# separate MW and TR courses
MW_courses = courses[courses['Days'] == 'MW']
TR_courses = courses[courses['Days'] == 'TR']

# for testing
# MW_courses = MW_courses[:120]
MW_courses.sort_values(by='Start Time', inplace=True)

# reset index
MW_courses.reset_index(drop=True, inplace=True)
TR_courses.reset_index(drop=True, inplace=True)

C:\Users\usama\AppData\Local\Temp\ipykernel_25516\3773065086.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MW_courses.sort_values(by='Start Time', inplace=True)


In [68]:
# helper functions

# max utilization ratio
def w1(i, j):
    return MW_courses['Enrollment'][i] / available_venues['Seating Capacity'][j]

# final weight
def weighted_product(i, j, x):
    return w1(i, j)

In [71]:
problem = LpProblem("Class-Scheduling", LpMaximize)

# each course can be scheduled in one of the available venues
x = LpVariable.dicts("x", [(i, j) for i in range(MW_courses.shape[0]) for j in range(len(available_venues))], 0, 1, LpBinary)

# objective function
problem += lpSum(x[i, j] * weighted_product(i, j, x) for i in range(MW_courses.shape[0]) for j in range(len(available_venues)))

# Constraint: Each course is scheduled in exactly one venue
for i in range(MW_courses.shape[0]):
    problem += lpSum(x[i, j] for j in range(len(available_venues))) == 1

# Constraint: Assigned venue has enough seating capacity
for i in range(MW_courses.shape[0]):
    for j in range(len(available_venues)):
        if MW_courses['Enrollment'][i] > available_venues['Seating Capacity'][j]:
            problem += x[i, j] == 0

# Constraint: No two courses are scheduled in the same venue at the same time
overlapping_slots = []
for i in range(MW_courses.shape[0]):
    for k in range(i, MW_courses.shape[0]):
        a = MW_courses['Start Time'][i]
        b = MW_courses['End Time'][i]
        c = MW_courses['Start Time'][k]
        d = MW_courses['End Time'][k]

        # add all overlaps to a list
        if a < c and c < b and b < d:
            overlapping_slots.append((i, k))

        if a < c and c < d and d < b:
            overlapping_slots.append((i, k))
            


    # overlaps = [k for k in range(i, MW_courses.shape[0]) if MW_courses['Start Time'][i] < MW_courses['End Time'][k] and MW_courses['Start Time'][k] < MW_courses['End Time'][i]]
    # if len(overlaps) > 1:
    #     overlapping_slots.append(overlaps)

# for j in range(len(available_venues)):
#     for overlap in overlapping_slots:
#         problem += lpSum(x[i, j] for i in overlap) <= 1

for j in range(len(available_venues)):
    for overlap in overlapping_slots:
        problem += lpSum(x[overlap[0], j] + x[overlap[1], j]) <= 0

In [72]:
# # Constraint: No overlapping courses in the same venue
# overlapping_slots = set([])
# for i in range(MW_courses.shape[0]):
#     overlap = [overlapping_slots.add((i, k)) for k in range(MW_courses.shape[0]) if i!=k and (MW_courses['Start Time'][i] < MW_courses['End Time'][k] and MW_courses['End Time'][i] > MW_courses['Start Time'][k])]

# for j in range(len(available_venues)):
#     # print(MW_courses['Course Code'][i], [MW_courses['Course Code'][k] for k in overlaps])
#     for overlap in overlapping_slots:
#         problem += x[overlap[0], j] + x[overlap[1], j] <= 1

In [73]:
print(overlapping_slots)
# for i in range(5):
#     print(overlapping_slots[i])
    # print([f"{MW_courses['Start Time'][j]}-{MW_courses['End Time'][j]}" for j in overlapping_slots[i]])

print(len(overlapping_slots))

[(0, 21), (0, 22), (0, 23), (0, 24), (0, 25), (0, 26), (0, 27), (0, 28), (0, 29), (0, 30), (0, 31), (0, 32), (0, 33), (0, 34), (0, 35), (0, 36), (0, 37), (0, 38), (0, 39), (0, 40), (1, 21), (1, 22), (1, 23), (1, 24), (1, 25), (1, 26), (1, 27), (1, 28), (1, 29), (1, 30), (1, 31), (1, 32), (1, 33), (1, 34), (1, 35), (1, 36), (1, 37), (1, 38), (1, 39), (1, 40), (2, 21), (2, 22), (2, 23), (2, 24), (2, 25), (2, 26), (2, 27), (2, 28), (2, 29), (2, 30), (2, 31), (2, 32), (2, 33), (2, 34), (2, 35), (2, 36), (2, 37), (2, 38), (2, 39), (2, 40), (3, 21), (3, 22), (3, 23), (3, 24), (3, 25), (3, 26), (3, 27), (3, 28), (3, 29), (3, 30), (3, 31), (3, 32), (3, 33), (3, 34), (3, 35), (3, 36), (3, 37), (3, 38), (3, 39), (3, 40), (3, 41), (3, 42), (3, 43), (3, 44), (4, 21), (4, 22), (4, 23), (4, 24), (4, 25), (4, 26), (4, 27), (4, 28), (4, 29), (4, 30), (4, 31), (4, 32), (4, 33), (4, 34), (4, 35), (4, 36), (4, 37), (4, 38), (4, 39), (4, 40), (5, 21), (5, 22), (5, 23), (5, 24), (5, 25), (5, 26), (5, 27), 

In [74]:
len(problem.constraints)

136069

In [75]:
# # # CAUTION !!! (approx. 1 hour runtime) # # #
problem.solve(PULP_CBC_CMD())

-1

In [76]:
# print the results
print("Course Code, Start Time, End Time, Venue, Enrollment, Seating Capacity")
for i in range(MW_courses.shape[0]):
    for j in range(len(available_venues)):
        if x[i, j].value() == 1:
            print(f"{MW_courses['Course Code'][i]}, {MW_courses['Start Time'][i]}, {MW_courses['End Time'][i]}, {available_venues['Class Venues'][j]}, {MW_courses['Enrollment'][i]}, {available_venues['Seating Capacity'][j]}")

# # classes scheduled per venue
# print("\nVenue, Classes Scheduled")
# for j in range(len(available_venues)):
#     print(f"{available_venues['Class Venues'][j]}, {sum(x[i, j].value() for i in range(MW_courses.shape[0]))}")

Course Code, Start Time, End Time, Venue, Enrollment, Seating Capacity
CHEM 342-LEC 1, 8:00 AM, 9:15 AM, SBASSE 10-204, 40, 40
FINN 200-LEC 2, 8:00 AM, 9:15 AM, Acad Block A-1, 55, 55
MATH 341-LEC 1, 8:00 AM, 9:15 AM, SBASSE 10-202, 50, 50
CLCA 3325-LEC 1, 8:00 AM, 9:50 AM, SBASSE 10-204, 37, 40
SS 100-LEC 13, 8:00 AM, 9:50 AM, Acad Block A-6, 25, 28
ENGL 2402-LEC 1, 8:00 AM, 9:50 AM, SBASSE 10-202, 45, 50
MGMT 260-LEC 1, 8:00 AM, 9:15 AM, Acad Block A-1, 55, 55
DISC 112-LEC 1, 8:00 AM, 9:15 AM, Acad Block A-1, 55, 55
ENGL 3172-LEC 1, 8:00 AM, 9:50 AM, SBASSE 10-302, 30, 30
SS 100-LEC 15, 9:00 AM, 10:50 AM, Acad Block A-6, 25, 28
EE 5502-LEC 1, 9:00 AM, 10:15 AM, SBASSE 10-202, 50, 50
CHE 330-LEC 1, 9:00 AM, 10:15 AM, SBASSE 10-204, 40, 40
EE 5502-LEC 1, 9:00 AM, 10:15 AM, SBASSE 10-202, 50, 50
CHEM 721-LEC 1, 9:00 AM, 10:15 AM, Acad Block A-6, 20, 28
ECON 4410-LEC 1, 9:00 AM, 10:50 AM, SBASSE 10-204, 40, 40
ECON 4410-LEC 1, 9:00 AM, 10:50 AM, SBASSE 10-204, 40, 40
CS 340-LEC 1, 9:00 A

In [77]:
import pandas as pd

MW_courses = pd.DataFrame({
    "Start Time": [8.0, 9.0, 10.0, 9.5, 12.0, 13.5],  # Start times of courses
    "End Time": [9.0, 10.5, 11.0, 11.0, 13.0, 14.0],   # End times of courses
})

for i in range(MW_courses.shape[0]):
    overlaps = [
        k
        for k in range(MW_courses.shape[0])
        if i != k and (
            MW_courses["Start Time"][i] < MW_courses["End Time"][k]
            and MW_courses["End Time"][i] > MW_courses["Start Time"][k]
        )
    ]
    print(f"Course {i}: Start={MW_courses['Start Time'][i]}, End={MW_courses['End Time'][i]}")
    print(f"Overlaps: {overlaps}")
    print("-" * 40)


Course 0: Start=8.0, End=9.0
Overlaps: []
----------------------------------------
Course 1: Start=9.0, End=10.5
Overlaps: [2, 3]
----------------------------------------
Course 2: Start=10.0, End=11.0
Overlaps: [1, 3]
----------------------------------------
Course 3: Start=9.5, End=11.0
Overlaps: [1, 2]
----------------------------------------
Course 4: Start=12.0, End=13.0
Overlaps: []
----------------------------------------
Course 5: Start=13.5, End=14.0
Overlaps: []
----------------------------------------
